In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v1 as tf

from glob import glob
from re import findall,sub
from tqdm.notebook import tqdm

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  *
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

%matplotlib inline

/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/codekage/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
tf.enable_eager_execution()

In [3]:
tf.config.experimental_list_devices()

['/job:localhost/replica:0/task:0/device:CPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_GPU:0',
 '/job:localhost/replica:0/task:0/device:XLA_CPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
images = glob("./lfw_funneled/*/*")

In [5]:
people = np.array([[" ".join(i.split("/")[-1].split("_")[:-1]),i] for i in images])

### Counting Occurence Of A Person

In [6]:
from collections import Counter
from random import choice

import pandas as pd

In [7]:
face_occurence = Counter(people[:,0])

In [8]:
face_occurence = pd.DataFrame([{"name":i,"occurence":face_occurence[i]} for i in face_occurence])

In [9]:
face_occurence.head()

,name,occurence
0,Dita Von Tesse,1
1,Chris Byrd,2
2,Irfan Ahmed,1
3,John Edwards,8
4,Manfred Reyes Villa,1


In [25]:
haar = cv2.CascadeClassifier()
haar.load("../haarcascade_frontalface_default.xml")

True

In [26]:
names = []
images = []
failed = []

for name,path in tqdm(people):
    img = cv2.imread(path)
    try:
        gray = img.mean(axis=2).astype(np.uint8)
        x,y,w,h = haar.detectMultiScale(gray)[0]
        img = cv2.resize(img[y:y+h,x:x+w],(128,128))
        names.append(name)
        images.append(img)
    except Exception as e:
        failed.append([name,img,e])
    
images = np.array(images)
names = np.array(names)

np.save("./names.npy",names)
np.save("./faces.npy",images)

In [29]:
names = np.load("./names.npy")
images = np.load("./faces.npy").astype(np.float32)

In [30]:
net = keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet',include_top=False,input_shape=(128,128,3),pooling=None)

In [39]:
for layer in net.layers:
    layer.trainable = True

In [75]:
model = keras.Sequential([
    net,
    keras.layers.Conv2D(4096,1,activation="relu"),
    keras.layers.Flatten(),
    keras.layers.Dense(2048,activation="relu"),
    keras.layers.Dense(1024,activation="relu"),
    keras.layers.Dense(512,activation="linear")
])

In [88]:
images.shape

(13190, 128, 128, 3)

In [93]:
pairs = []

for name in set(names):
    group = np.where(name == names)[0]
    for p in group:
        pairs.append([
            p,
            choice(group),
            np.random.randint(13190)
        ])

pairs = np.array(pairs)